# CAUTION: This page is extremely long as it contains print outputs that are 2000+ lines, and scrolling through the page can cause lag. This was primarily a workbook that's not formatted for external viewing.

# You can see only the [.py code here](https://github.com/chanvarma/thinkful-ds/blob/master/capstones/1_books_to_movies/data_munging_pipeline.py), and read more about the [ETL pipeline here](https://github.com/chanvarma/thinkful-ds/blob/master/capstones/1_books_to_movies/README.md). Thanks!   

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-statements-and-API-keys" data-toc-modified-id="Import-statements-and-API-keys-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import statements and API keys</a></span></li><li><span><a href="#Information-sources" data-toc-modified-id="Information-sources-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Information sources</a></span></li><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper functions</a></span></li><li><span><a href="#Functions-that-webscrape-(no-explicit-API-calls)" data-toc-modified-id="Functions-that-webscrape-(no-explicit-API-calls)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Functions that webscrape (no explicit API calls)</a></span></li><li><span><a href="#Functions-with-explicit-API-calls" data-toc-modified-id="Functions-with-explicit-API-calls-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Functions with explicit API calls</a></span></li><li><span><a href="#Start-here" data-toc-modified-id="Start-here-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Start here</a></span><ul class="toc-item"><li><span><a href="#Parse-wikipedia-articles-to-build-dictionaries" data-toc-modified-id="Parse-wikipedia-articles-to-build-dictionaries-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Parse wikipedia articles to build dictionaries</a></span></li><li><span><a href="#Data-after-Lv-1-enrichment" data-toc-modified-id="Data-after-Lv-1-enrichment-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Data after Lv 1 enrichment</a></span></li><li><span><a href="#Get-IMDB-numbers-for-movies-provided-by-Wikipedia" data-toc-modified-id="Get-IMDB-numbers-for-movies-provided-by-Wikipedia-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Get IMDB numbers for movies provided by Wikipedia</a></span></li><li><span><a href="#Scrape-IMDb-lists-to-add-new-movies-and-books-to-dictionaries" data-toc-modified-id="Scrape-IMDb-lists-to-add-new-movies-and-books-to-dictionaries-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Scrape IMDb lists to add new movies and books to dictionaries</a></span></li><li><span><a href="#De-facto-data-size" data-toc-modified-id="De-facto-data-size-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span><em>De facto</em> data size</a></span></li></ul></li><li><span><a href="#Enrich-data-using-APIs" data-toc-modified-id="Enrich-data-using-APIs-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Enrich data using APIs</a></span><ul class="toc-item"><li><span><a href="#Get-book-relatated-metadata-from-OpenLibrary-API-and-Goodreads-APIs" data-toc-modified-id="Get-book-relatated-metadata-from-OpenLibrary-API-and-Goodreads-APIs-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Get book relatated metadata from OpenLibrary API and Goodreads APIs</a></span></li><li><span><a href="#Get-book-relatated-metadata-from-IMDb-API" data-toc-modified-id="Get-book-relatated-metadata-from-IMDb-API-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Get book relatated metadata from IMDb API</a></span></li></ul></li><li><span><a href="#Create-unique-pairing-IDs-for-every-movie-and-book" data-toc-modified-id="Create-unique-pairing-IDs-for-every-movie-and-book-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Create unique pairing IDs for every movie and book</a></span><ul class="toc-item"><li><span><a href="#Deep-copy-dictionaries" data-toc-modified-id="Deep-copy-dictionaries-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Deep copy dictionaries</a></span></li></ul></li><li><span><a href="#Data-enrichment-complete!" data-toc-modified-id="Data-enrichment-complete!-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Data enrichment complete!</a></span></li><li><span><a href="#Converting-to-dataframes" data-toc-modified-id="Converting-to-dataframes-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Converting to dataframes</a></span><ul class="toc-item"><li><span><a href="#Export-to-CSV" data-toc-modified-id="Export-to-CSV-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Export to CSV</a></span></li></ul></li></ul></div>

## Import statements and API keys

In [19]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import copy                               #For deep copying dicts to create 'save points'

from goodreads import client              #For adding book related metadata 
gc = client.GoodreadsClient('9addHIFaPJmit7dzC5ZA', 'iDXvx13Rnt1iJkWOxRkSh9wLnbFWmMNgCUMFLJmLPo')

from imdb import IMDb                     #For adding book related metadata  
OMBD_API_KEY = '2cb213a7'

from xml.parsers.expat import ExpatError  #For error handling while working with links
import socket  

In [155]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

## Information sources

In [20]:
wikipages = ['https://en.wikipedia.org/wiki/List_of_children%27s_books_made_into_feature_films?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)?oldformat=true',
             'https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)?oldformat=true'
            ]

In [21]:
imdb_lists = ['https://www.imdb.com/list/ls000989358/', #2011 list: 71 movies 
              'https://www.imdb.com/list/ls000004341/', #2010 list: 20 movies 
              'https://www.imdb.com/list/ls070762787/', #2014 list: 20 movies 
              'https://www.imdb.com/list/ls029848347/', #2019 list: 23 movies 
              'https://www.imdb.com/list/ls053974803/', #2013 list: 23 movies
              'https://www.imdb.com/list/ls076239083/', #2015 list: 10 movies 
              'https://www.imdb.com/list/ls000420337/?sort=user_rating,desc&st_dt=&mode=detail&page=1', 
              #2011 list: 100 movies, sorted by IMDb ratings (desc)
              'https://www.imdb.com/list/ls000099866/?sort=runtime,desc&st_dt=&mode=detail&page=1', 
              #2011 list: 100 movies, sorted by Runtime (desc)
            ]

## Helper functions

In [23]:
def working_wiki_link(wiki_link):
    """
    Converts relative URLs to global URLs.
    """
    if 'fr.wiki' in wiki_link or 'ru.wiki' in wiki_link:
            return ''
    
    if '/wiki/' in wiki_link:
        return 'https://en.wikipedia.org' + wiki_link
    return wiki_link

In [24]:
def get_adaptations(book_title, href_list, isbn_oclc):
    """
    Parses through list of movies associated with a specific `book_title`,
    adds movies to `movie_book_dict`, and returns list of movies.
    Adds year to the movie title if information available. 
    
    Args:
        book_title: (str) name of original book.
        href_list: (list) Result of findAll('a') on the movies column.
        isbn_oclc: (dict) ISBN and OCLC numbers to pass on to movie.
        
    Returns:
        adaptations: List of movie title names.
    """
    adaptations = []
    
    if len(href_list) == 1:
        adaptations.append(href_list[0]['title'])
        return adaptations
        
    for index, tag in enumerate(href_list):
        year_search = re.search(r'\d\d\d\d', tag['href'])
        
        if year_search is not None:  #There is a 4 digit number in the title
            year_search = year_search.group()
        
            if year_search not in tag['title']:
                version = tag['title'] + ' (' + year_search + ')'
            else:
                version = tag['title']
            
            adaptations.append(version)
            
        
        else:
            version = tag['title']
            adaptations.append(version)
            
        try:
            isbn = isbn_oclc['isbn']
            oclc = isbn_oclc['oclc']
        except Exception:
            isbn = 'broken'
            oclc = 'broken'
            
        
        movie_book_dict[working_wiki_link(tag['href'])] = {'movie_title': version, 
                                                           'book_title': book_title,
                                                           'imdb_no': '', 'usable': '',
                                                           'isbn': isbn, 'oclc': oclc}

    
    return adaptations

In [25]:
def clean_adaptations(adaptations):
    """
    Makes sure the list of adaptations doesn't contain TV series or miniseries.
    """
    
    cleaned = []
    
    for title in adaptations:
        if 'TV' in title or 'miniseries' in title:
            continue
        elif 'page does not exist' in title:
            continue
        elif 'fr.wiki' or 'ru.wiki' in title:
            continue
        else:
            cleaned.append(title)
            
    return cleaned

In [26]:
def add_to_book_film_dict():
    """
    Method parses newly updated rows for book/film title, book author,
    number of adaptions, and list of adaptations, and adds them to the
    `book_film_dict` as a nested dictionary. 
    global sum_adaptations
    """
    global sum_adaptations
    
    for row in rows:
        cols = row.findAll('td')

        try:
            if (len(cols[0].findAll('a')) == 2) and (cols[1].find('a') is not None):
                series = cols[0].text.replace('\n', '').split(',')[0]

                book_title = cols[0].findAll('a')[0]['title']
                author = cols[0].findAll('a')[-1]['title']

                book_wiki_url = working_wiki_link(cols[0].findAll('a')[0]['href'])
                
                try:
                    isbn_oclc = get_isbn_oclc(book_wiki_url)
                    isbn = isbn_oclc['isbn']
                    oclc = isbn_oclc['oclc']
                except TypeError or KeyError:
                    isbn = 'broken'
                    oclc = 'broken'
                
                try:
                    adaptations = clean_adaptations(get_adaptations(book_title, cols[1].findAll('a'), isbn_oclc))
                except KeyError or TypeError:
                    adaptations = []
                    pass

                sum_adaptations += len(adaptations)

                if (isbn == 'broken' and oclc == 'broken'):  
                    print('{} by {} 𝙓'.format(book_title, author))
                else:
                    print('{} by {} ✔️'.format(book_title, author))
                
                book_movie_dict_all[series] = {'author': author,
                                            'book_title': book_title,
                                            'book_wiki_url': book_wiki_url,
                                            'count': len(adaptations),
                                            'adaptations': adaptations,
                                            'isbn': isbn, 'oclc': oclc, 'valid_identifier': ''}

        except IndexError or KeyError:
            pass
        
    print("No. of rows/original books in dict: {}, No. of adaptations: {} \n"
          .format(len(book_movie_dict_all), sum_adaptations))

In [235]:
def add_valid_books_to_dict():
    """
    Adds rows from `book_movie_dict_all` to `book_movie_dict'
    if they return a valid output for get_isbn_oclc(). 
    """
    global usable_count
    global broken_count
    
    for index, key in enumerate(book_movie_dict_all.keys()):
        link = book_movie_dict_all[key]['book_wiki_url']

        try:
            if book_movie_dict_all[key]['isbn'] == 'broken' and book_movie_dict_all[key]['oclc'] == 'broken':
                book_movie_dict_all[key]['valid_identifier'] = False
                    #Nothing changes, the book is unusable, and the 'usable' flag remains False
            else:
                book_movie_dict_all[key]['valid_identifier'] = True   
                book_movie_dict[key] = copy.deepcopy(book_movie_dict_all[key]) #Deepcopy 'true' values to another dictionary
                usable_count += 1
        except NameError:
            pass
        
        if (index % 100 == 0):
            broken_count = index - usable_count
            print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, index))
    
    print("Usable books: {}, Broken books: {}, Total books: {}".format(usable_count, broken_count, len_initial))

In [29]:
def add_wikipages_to_dicts():
    """
    Parses all the wikipedia links and adds valid books to `book_movie_dict`,
    and valid movies to `movie_book_dicts`.
    """
    global len_initial
    
    for url in wikipages:
        page = requests.get(url).text
        soup = BeautifulSoup(page, 'lxml')
    
        add_to_rows(soup)
        add_to_book_film_dict()

In [52]:
from socket import gaierror

def get_existing_movie_imdb_nos():
    """
    Iterates through `movie_book_dict` and fills in IMDb numbers
    from the wikipedia pages of the respective movie.
    """
    usable_count = 0
    
    errors = (requests.exceptions.MissingSchema or 
              socket.gaierror or 
              KeyError or 
              requests.exceptions.ConnectionError or 
              requests.exceptions.NewConnectionError or 
              requests.exceptions.MaxRetryError or Exception)

    for index, key in enumerate(movie_book_dict.keys()):
        movie_book_dict[key]['usable'] = False #default is false
        
        book_title = movie_book_dict[key]['book_title']
        if book_title not in book_movie_dict.keys():
            try:
                movie_book_dict[key]['imdb_no'] = get_imdb_no(key)
            except: 
                movie_book_dict[key]['imdb_no'] = 'broken'

            if movie_book_dict[key]['imdb_no'] != 'broken':
                movie_book_dict[key]['usable'] = True
                print('{} ✔️'.format(book_title))
                usable_count += 1
            else:
                print('{} 𝙓'.format(book_title))
                

            if (index + 1) % 150 == 0:
                print('Index: {}, Usable: {} --> {}%'
                      .format(index + 1, usable_count, round(usable_count*100/(index+1), 3)))    
        else:
            movie_book_dict[key]['imdb_no'] = 'broken'

In [33]:
def add_all_imdb_lists(imdb_lists):
    """
    Iterates through `imdb_lists` and passes each indivdual
    list to add_imdb_list().
    """
    global imdb_updates
    
    for link in imdb_lists:
        add_imdb_list(link)
        print('\n List done. Total added: {}'.format(imdb_updates))

In [34]:
from xml.parsers.expat import ExpatError

metadata_update_count = 0
        
def book_metadata_refresh():
    """
    Adds book metadata to entire dict.
    """
    
    global metadata_update_count

    for index, key in enumerate(book_movie_dict.keys()):
        update_book_metadata(key)
        
        if book_movie_dict[key]['metadata_updated'] is True:
            metadata_update_count += 1
            print('{} ✔'.format(key))
        else:
            print('{} 𝙓'.format(key))

        if index % 75 == 0 and index != 0:
            print("Indexed: {}, Updated: {}, Usable: {}%"
                  .format(index, metadata_update_count, round(100*metadata_update_count/index, 3)))    

In [35]:
def get_imdb_actors(actor_string):
    actors = actor_string.rstrip('.').split(',')
    actors = [a.lower().strip() for a in actors]    
    return actors

def get_imdb_writers(writer_string):
    writers = writer_string.rstrip('.').split(',')
    writers = [re.sub(r'\([^()]*\)', '', w) for w in writers]
    writers = [w.lower().strip() for w in writers] 
    return writers

def get_imdb_genres(genre_string):
    genres = genre_string.rstrip('.').split(',')
    genres = [g.lower().strip() for g in genres]    
    return genres

In [37]:
def movie_metadata_refresh():
    """
    Updates move metdata for all 'usable' movies in the movie_book_dict.
    """
    
    updated_count = 0
    for index, key in enumerate(movie_book_dict.keys()):

        if movie_book_dict[key]['usable'] is True:
            try:
                add_imdb_metadata(key)
            except json.JSONDecodeError:
                movie_book_dict[key]['usable'] = False
                movie_book_dict[key]['imdb_no'] = 'broken'
                updated_count -= 1
                pass

            print('{} ✔'.format(movie_book_dict[key]['movie_title']))
            updated_count += 1
        else:
            print('{} 𝙓'.format(movie_book_dict[key]['movie_title']))

        if index % 150 == 0 and index != 0:
            print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, round(updated_count * 100/index), 3))

    print('Index: {}, Updated: {} --> {}%'.format(index, updated_count, round(updated_count * 100/index), 3))

In [163]:
def match_ids():
    """
    Creates a book_id for each book/movie, to uniquly identify a book movie pair. 
    book_id = b + '_' + ISBN + '_'+ OCLC
    
    Creates a movie_id for each movie, to uniquly identify a book movie pair. 
    movie_id = m + '_' ISBN + '_'+ OCLC + '_' + 'imdb_no'
    
    """
    for key in book_movie_dict.keys():
        book_movie_dict[key]['book_id'] = 'b_' + book_movie_dict[key]['isbn'] + '_' + book_movie_dict[key]['oclc']
    print('Books updated.')
        
    for key in movie_book_dict.keys():
        movie_book_dict[key]['book_id'] = 'b_' + movie_book_dict[key]['isbn'] + '_' + movie_book_dict[key]['oclc']
                
        movie_book_dict[key]['movie_id'] = 'm_' + movie_book_dict[key]['isbn'] + '_' + movie_book_dict[key]['oclc'] + '_' + movie_book_dict[key]['imdb_no']
    print('Movies updated.')
    

## Functions that webscrape (no explicit API calls)

In [22]:
def add_to_rows(page_soup):
    """
    Method extracts row elements from the soup tags of individual 
    list pages (A - F, H - J, etc.) and appends them to the `rows` 
    list.
    
    Args:
        page_soup: (BeautifulSoup tag) soup for list page.
    """
    
    global rows
    
    tables = page_soup.find_all('table', {'class': 'wikitable'})

    for alphabet in tables:
        rows += alphabet.findAll('tr')
    
    print("Tables added: {}, Rows added: {}".format(len(tables), len(rows)))

In [28]:
def get_isbn_oclc(wiki_url):
    """
    Parses the `infobox vcard` element of the wikipedia page of a book, 
    and returns the 10-digit or 13-digit ISBN or 9-digit OCLC number (if found). 
    
    Args:
        wiki_url (str): Valid (not local) wikipedia link.

    Returns:
        (dictionary) = {'isbn': isbn, 'oclc': oclc}
        
        isbn: Returns ISBN code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. The code is de-hyphentated for later pipeline
        efficiency. 
        
        oclc: Returns OCLC code is parsed correctly, 'broken' if formmated 
        incorrectly or otherwise. 
 
    """
    try:
        page = requests.get(wiki_url).text
        soup = BeautifulSoup(page, 'lxml')
    except requests.exceptions.MissingSchema or Exception:
        return 'broken'  
    
    infobox = soup.find('table', {'class': 'infobox vcard'})
    isbn, oclc = '', ''
    
    if infobox is None:
        isbn, oclc = 'broken', 'broken'
    
    else:
        isbn_found, oclc_found = False, False
    
        for row in infobox.findAll('tr'):
        #ISBN directly available
            if 'ISBN' in row.text and isbn_found is False:
                isbn = re.sub('[^0-9]','', row.findAll('a')[-1].text)
                isbn_found = True
        
            if row.find('a', {'title': 'OCLC'}) is not None and oclc_found is False:
                oclc = row.find('td').text
                oclc = oclc.split()[0]
                oclc_found = True
    
        if isbn_found is False and oclc_found is False:
            isbn, oclc = 'broken', 'broken' 
        
    return {'isbn': isbn, 'oclc': oclc}    

In [30]:
def get_imdb_no(wiki_movie):
    """
    Returns movie's respective IMDB no (if found). 
    
    Args: 
        wiki_movie: (str) Wikipedia link of movie's article.
        
    Returns:
        imdb_no: (str) IMDB no. if found, 'broken' otherwise.
    """
    imdb_no = ''   
   
    errors = (requests.exceptions.MissingSchema or 

          KeyError or 
          requests.exceptions.ConnectionError or 
          requests.exceptions.NewConnectionError or 
          requests.exceptions.MaxRetryError)
    
    try:
        page = requests.get(wiki_movie).text
        soup = BeautifulSoup(page, 'lxml')
    except errors:
        return 'broken'
    
    external_links = soup.find('a', href=re.compile("imdb.com/title"))
    
    try:
        imdb_no = external_links['href'].split('/')[4]
    except TypeError:
        return 'broken'
    
    return imdb_no  

In [32]:
def add_imdb_list(imdb_list):
    """
    Adds books and movies from 1 imdb list.
    
    Args:
        imdb_lists: (list) Link to individual imdb list.
    """
    global imdb_updates
    
    new_books = {}
    new_movies = {}
    
    page = requests.get(imdb_list).text
    soup = BeautifulSoup(page, 'lxml')
    print('Working on: {} | {}'.format(soup.title.text, imdb_list))
    
    list_elements = soup.findAll('h3', {'class': 'lister-item-header'})
    list_elements = [l.find('a') for l in list_elements]
    no_elements = len(list_elements)
    
    for index, l in enumerate(list_elements):
        imdb_no = l['href'].split('/')[2]
        imdb_link = 'https://imdb.com' + l['href']
        
        movie_title = l.text
        
        #Get details about new books
        book_essentials = get_book_essentials_from_movie_title(movie_title)
        new_books[book_essentials['book_title']] = book_essentials
        
        #Get details about new movies
        movie_essentials = {'movie_title': movie_title, 
                            'book_title': book_essentials['book_title'],
                            'imdb_no': imdb_no,
                            'isbn': book_essentials['isbn'],
                            'oclc': book_essentials['oclc'],
                            'usable': True}
        new_movies[imdb_link] = movie_essentials
        
        #Adds newly sources books and movies to initial dictionaries
        book_movie_dict.update(new_books)
        movie_book_dict.update(new_movies)
        
        imdb_updates += 1
        
        if imdb_updates % 50 == 0:
            try:
                print('Added from this list: {}, Total in this list: {}, Total added: {}'.format(index, no_elements))
            except IndexError:
                pass
    
    return

In [36]:
def get_box_office(imdb_no):
    """
    Scrapes a particular IMDB page for box office information.
    """
    
    imdb_page = 'https://www.imdb.com/title/' + imdb_no
    page = requests.get(imdb_page).text
    soup = BeautifulSoup(page, 'lxml')
    
    wanted = ['Budget', 'Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross']
    box_office_stats = {}
    
    for div in soup.findAll('div', {'class': 'txt-block'}):
        try:
            subheading = div.find('h4').text.split(':')[0]
        except AttributeError:
            subheading = None
            pass
        
        if subheading in wanted:
            try:
                money = div.text.split(':')[1].split('\n')[0]
                money = money.replace(',', '').split()[0].split('$')[1]
            except IndexError or KeyError:
                money = 'N/A'
            
            subheading = subheading.lower().replace(' ', '_')
            box_office_stats[subheading] = money
    
    return box_office_stats

get_box_office('tt0454876')

{'budget': '120000000',
 'opening_weekend_usa': '22451514',
 'gross_usa': '124987023',
 'cumulative_worldwide_gross': '609016565'}

## Functions with explicit API calls

In [81]:
book_metadata_wanted = ['title', 'identifiers', 'number_of_pages', 'subject_places', 
                        'subjects', 'publish_date', 'publish_places', 'author']

def update_book_metadata(book_title):
    """
    Uses the OpenLibrary and Goodreads APIs (when possible) to 
    add metadata about the books to the `book_movie_dict`.
    
    Args: 
        book_title: (str) used when merging back into `book_movie_dict`.
    
    Returns:
        updated: (bool) True if metadata updated, False otherwise.
    
    
    """
    metadata = {}
    all_metadata = {}
    
    
    if book_movie_dict[book_title]['valid_identifier'] is False:
        #API can't be accessed
        metadata['metadata_updated'] = False
        book_movie_dict[book_title].update(metadata)
        return
    
    elif book_movie_dict[book_title]['isbn'] != '':
        code = book_movie_dict[book_title]['isbn']
        curl = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
    else:
        code = book_movie_dict[book_title]['oclc']
        curl = 'https://openlibrary.org/api/books?bibkeys=OCLC:{}&jscmd=data&format=json'.format(code)
        all_metadata = requests.get(curl).json()
        
    if len(all_metadata) > 0:
        key = list(all_metadata.keys())[0]
        nested_keys = list(all_metadata[key].keys())
        
        for data_point in book_metadata_wanted:
            if data_point in nested_keys:
                metadata[data_point] = all_metadata[key][data_point]
    
        if 'subject_places' in nested_keys:
            metadata['subject_places'] = [place['name'][:30] for place in metadata['subject_places']]
        if 'subjects' in nested_keys:
            metadata['subjects'] = [subject['name'][:30] for subject in metadata['subjects']]
        if 'publish_places' in nested_keys:
            metadata['publish_places'] = [place['name'][:30] for place in metadata['publish_places']]            
        
        if 'identifiers' in nested_keys:
            if 'goodreads' in list(metadata['identifiers'].keys()):
                metadata['goodreads'] = metadata['identifiers']['goodreads'][0]
            metadata.pop('identifiers')
            
        if 'goodreads' in metadata.keys():
            metadata.update(get_goodreads_data(metadata['goodreads']))
            
        metadata['metadata_updated'] = True
        book_movie_dict[book_title].update(metadata)
        return
    
    else:
        metadata['metadata_updated'] = False
        book_movie_dict[book_title].update(metadata)
        return

In [85]:
desired_metadata = ['average_rating', 
                    'publication_date',
                    'rating_dist',
                    'ratings_count', 'format',
                    'text_reviews_count',
                    'title', 'language_code', 'author', 'authors']

def get_goodreads_data(gc_code):
    """
    Add additional reviews and ratings related meta_data from the GoodReads API.
    
    Args:
        gc_code: Strictly Goodreads code. Can be 10-digit or 13-digit.
        
    Returns:
        gr_metadata: Dict with additional metadata.
    """
    
    gr_metadata = {}
    try:
        book = gc.book(gc_code)
    except NameError:
        return gr_metadata
    except ExpatError:
        return gr_metadata
    
    for attrs in book.__dict__['_book_dict']:
        if attrs in desired_metadata:
            gr_metadata['gc_' + attrs] = book.__dict__['_book_dict'][attrs]
            
    return gr_metadata

In [41]:
movie_metadata_wanted = ['Title', 'Year', 'Rated', 'Released', 'Runtime', 
                         'English', 'imdbRating', 'imdbVotes']

def add_imdb_metadata(movie_url):
    """
    Adds IMDB metadata from the OMBD API.
    
    Args:
        movie_url: (str) essentially, the key for each movie in `movie_book_dict`. 
                    Can be wikipedia link or imdb link.
    """
    metadata = {}
    all_metadata = {}
    imdb_no = movie_book_dict[movie_url]['imdb_no']
    
    all_metadata = requests.get('http://www.omdbapi.com/?i={}&apikey=2cb213a7'.format(imdb_no)).json()
    
    for key in all_metadata.keys():
        if key in movie_metadata_wanted:
            metadata['imdb_' + key.lower()] = all_metadata[key]
            
        if 'Actors' in all_metadata.keys():
            metadata['imdb_actors'] = get_imdb_actors(all_metadata['Actors'])
        if 'Writer' in all_metadata.keys():
            metadata['imdb_writers'] = get_imdb_writers(all_metadata['Writer'])
        if 'Genre' in all_metadata.keys():
            metadata['imdb_genres'] = get_imdb_genres(all_metadata['Genre'])
            
    metadata.update(get_box_office(imdb_no))
            
    metadata['metadata_updated'] = True
    
    movie_book_dict[movie_url].update(metadata)
    

In [236]:
def get_book_essentials_from_movie_title(movie_title):
    """
    Reverse look up of a movie's wikipedia page to identify source book.
    
    Returns:
        book_essentials: (dict) of book detail in the `book_movie_dict` format. 
    """
    
    book_essentials = {}
    possible_wiki_combos = []
    isbn, oclc = '', ''
    valid_identifier = False
    
    # Assume that movie title is the book title
    book_title = movie_title
    wiki_link = 'https://en.wikipedia.org/wiki/' + book_title
    
    isbn_oclc = get_isbn_oclc(wiki_link)
    isbn = isbn_oclc['isbn']
    oclc = isbn_oclc['oclc']
               
    if isbn == 'broken' and oclc == 'broken':
        possible_wiki_combos.append(movie_title.replace('&', 'and'))
        possible_wiki_combos.append(movie_title + '_(novel)')
        possible_wiki_combos.append(movie_title + '_(book)')
 
        try:
            possible_wiki_combos.append(movie_title.split(':')[1].strip()) #Removes :
            possible_wiki_combos.append(movie_title.split(':')[1].split('-')[0].strip()) #Removes :
        except IndexError:
            pass
        
        for possible_title in possible_wiki_combos:
            wiki_link = 'https://en.wikipedia.org/wiki/' + possible_title
            isbn_oclc = get_isbn_oclc(wiki_link)
            isbn = isbn_oclc['isbn']
            oclc = isbn_oclc['oclc']
        
            if isbn != 'broken' or oclc != 'broken':
                book_title = possible_title
                valid_identifier = True
                break
            else:
                wiki_link = 'https://en.wikipedia.org/wiki/The ' + possible_title
                isbn_oclc = get_isbn_oclc(wiki_link)
                isbn = isbn_oclc['isbn']
                oclc = isbn_oclc['oclc']
        
                if isbn != 'broken' or oclc != 'broken':
                    book_title = possible_title
                    valid_identifier = True
                    break
    else:
        valid_identifier = True
                
    if valid_identifier is False:
        wiki_link, book_title = '', 'unknown'
    else:
        print('{} ✔'.format(book_title))

    book_essentials = {'author': '',
                       'book_title': book_title,
                       'book_wiki_url': wiki_link,
                       'count': 1,
                       'adaptations': [movie_title],
                       'isbn': isbn,
                       'oclc': oclc,
                       'valid_identifier': valid_identifier} 
    
    return (book_essentials)
    
get_book_essentials_from_movie_title("Percy Jackson: Sea of Monsters")

Sea of Monsters ✔


{'author': '',
 'book_title': 'Sea of Monsters',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/The Sea of Monsters',
 'count': 1,
 'adaptations': ['Percy Jackson: Sea of Monsters'],
 'isbn': '0786856866',
 'oclc': '64664383',
 'valid_identifier': True}

In [110]:
def clean_gc_authors():
    """
    Updates web-scraped authors with Goodreads info.
    """
    
    update = 0
    for index, key in enumerate(book_movie_dict.keys()):
        try:
            gc_authors_clean = book_movie_dict[key]['gc_authors']['author']['name']
            book_movie_dict[key]['author'] = gc_authors_clean
            print('{} ✔️'.format(key))
            
            del book_movie_dict[key]['gc_authors']
            update += 1
            
        except Exception:
            print('{} 𝙓'.format(key))
            pass
    print('Updated: {}, total: {}'.format(update, index))

## Start here

### Parse wikipedia articles to build dictionaries

In [43]:
rows = []
book_movie_dict_all = {}
movie_book_dict = {}
sum_adaptations = 0
len_initial = 0
broken_count = 0
usable_count = 0
book_movie_dict = {}
imdb_updates = 0

add_wikipages_to_dicts()

len_initial = len(book_movie_dict_all)

Tables added: 25, Rows added: 376
Adventures of Huckleberry Finn by Mark Twain ✔️
The Adventures of Pinocchio by Carlo Collodi 𝙓
The Adventures of Tom Sawyer by Mark Twain ✔️
Alexander and the Terrible, Horrible, No Good, Very Bad Day by Judith Viorst ✔️
Alice series by Phyllis Reynolds Naylor 𝙓
Alisa Selezneva by Kir Bulychev 𝙓
Angus, Thongs and Full-Frontal Snogging by Louise Rennison ✔️
Anne of Green Gables by Lucy Maud Montgomery 𝙓
The Apple Dumpling Gang by Jack Bickham 𝙓
Aquamarine (novel) by Alice Hoffman ✔️
Argument About Basia by Kornel Makuszyński 𝙓
Arrow to the Sun by Gerald McDermott ✔️
Arthur Read by Marc Brown (author) 𝙓
Ballet Shoes (novel) by Noel Streatfeild ✔️
Bambi, A Life in the Woods by Felix Salten ✔️
Basil of Baker Street by Eve Titus 𝙓
Beastly by Alex Flinn ✔️
Because of Winn-Dixie by Kate DiCamillo ✔️
Because of Winn-Dixie by Kate DiCamillo ✔️
Ben-Hur: A Tale of the Christ by Lew Wallace 𝙓
The BFG by Roald Dahl ✔️
Big Fish: A Novel of Mythic Proportions by Dani

Searching for David's Heart by Cherie Bennett ✔️
The Secret Garden by Frances Hodgson Burnett 𝙓
A Series of Unfortunate Events by Lemony Snicket 𝙓
The Sheep-Pig by Dick King-Smith ✔️
Shrek! by William Steig ✔️
Simon vs. the Homo Sapiens Agenda by Becky Albertalli ✔️
The Sisterhood of the Traveling Pants (novel) by Ann Brashares ✔️
Skellig by David Almond ✔️
The Sneetches and Other Stories by Dr. Seuss ✔️
The Snowman by Raymond Briggs ✔️
Someone like You (novel) by Sarah Dessen ✔️
Sounder by William H. Armstrong ✔️
Speak (Anderson novel) by Laurie Halse Anderson ✔️
Stardust (novel) by Neil Gaiman ✔️
Storm Boy (novel) by Colin Thiele 𝙓
Stormbreaker (novel) by Anthony Horowitz ✔️
The Story of Ferdinand by Munro Leaf 𝙓
The Story of Holly and Ivy by Rumer Godden 𝙓
Stuart Little by E. B. White 𝙓
The Stupids by James Marshall (author) ✔️
Swallows and Amazons by Arthur Ransome ✔️
The Sword in the Stone (novel) by T. H. White 𝙓
The Tale of Despereaux by Kate Dicamillo ✔️
A Tale of Two Cities by

James and the Giant Peach by Roald Dahl ✔️
Johnny Tremain by Esther Forbes ✔️
Jumanji by Chris Van Allsburg 𝙓
Kaytek the Wizard by Janusz Korczak ✔️
King Matt the First by Janusz Korczak ✔️
The King's Damosel by Vera Chapman ✔️
Lassie Come-Home by Eric Knight 𝙓
The Last Unicorn by Peter S. Beagle ✔️
The Legend of Sleepy Hollow by Washington Irving 𝙓
Little Lord Fauntleroy by Frances Hodgson Burnett 𝙓
A Little Princess by Frances Hodgson Burnett 𝙓
Little Men by Louisa May Alcott 𝙓
Little Toot by Hardie Gramatky ✔️
The Little White Horse by Elizabeth Goudge ✔️
Little Women by Louisa May Alcott 𝙓
The Littles by John Peterson (author) 𝙓
The Lorax by Dr. Seuss ✔️
Lottie and Lisa by Erich Kästner 𝙓
Madicken by Astrid Lindgren 𝙓
Madame Doubtfire by Anne Fine ✔️
The Marvelous Land of Oz by L. Frank Baum 𝙓
Mary Poppins (book series) by P. L. Travers 𝙓
Matilda (novel) by Roald Dahl 𝙓
Maximum Ride: The Angel Experiment by James Patterson ✔️
Me and Earl and the Dying Girl by Jesse Andrews ✔️
Midni

Angels & Demons by Dan Brown ✔️
Anglo-Saxon Attitudes by Angus Wilson 𝙓
The Angry Hills by Leon Uris 𝙓
Un animal doué de raison (page does not exist) by Robert Merle 𝙓
Animal Farm by George Orwell ✔️
Anna and the King of Siam (novel) by Margaret Landon 𝙓
Anna Karenina by Leo Tolstoy ✔️
Anne of Avonlea by Lucy Maud Montgomery 𝙓
Anne of Green Gables by Lucy Maud Montgomery 𝙓
The Antagonists by Ernest K. Gann 𝙓
Antonieta by Andrés Henestrosa 𝙓
The Apocalypse Watch by Robert Ludlum ✔️
The Apple Dumpling Gang by Jack Bickham 𝙓
Appointment with Death by Agatha Christie 𝙓
Appointment with Venus by Jerrard Tickell 𝙓
The Apprenticeship of Duddy Kravitz (novel) by Mordecai Richler ✔️
Armageddon 2419 A.D. by Philip Francis Nowlan 𝙓
Arrowsmith (novel) by Sinclair Lewis ✔️
Ask the Dust by John Fante ✔️
The Assassination Bureau, Ltd by Jack London ✔️
The Assassination of Jesse James by the Coward Robert Ford (novel) by Ron Hansen (novelist) ✔️
L'Assommoir by Émile Zola 𝙓
Assunta Spina (play) (page d

Ann Ireland by Ann Ireland 𝙓
Dany Laferrière by Dany Laferrière 𝙓
The Chamber (novel) by John Grisham ✔️
The Blue Room (novel) (page does not exist) by Georges Simenon 𝙓
Champagne (1928 film) by Walter C. Mycroft 𝙓
The Chancellor Manuscript by Robert Ludlum ✔️
Wang Anyi by Wang Anyi 𝙓
Charlie and the Chocolate Factory by Roald Dahl ✔️
Charlie Chan Carries On by Earl Derr Biggers ✔️
Charlie M by Brian Freemantle 𝙓
Charlotte Gray (novel) by Sebastian Faulks ✔️
Charlotte's Web by E. B. White ✔️
The Cheetah Girls (novel series) by Deborah Gregory 𝙓
The Children of Men by P. D. James ✔️
The Children of Sanchez (book) by Oscar Lewis 𝙓
China Seas (novel) (page does not exist) by Crosbie Garstin 𝙓
The Chinese Parrot by Earl Derr Biggers 𝙓
Chocolat (novel) by Joanne Harris ✔️
The Chocolate War by Robert Cormier ✔️
Christine (novel) by Stephen King ✔️
J. I. M. Stewart by J. I. M. Stewart 𝙓
Glendon Swarthout by Glendon Swarthout 𝙓
W. Somerset Maugham bibliography by W. Somerset Maugham 𝙓
Gilbert 

...

Blackbeard's Ghost by Ben Stahl (artist) 𝙓
Elisabeth Sanxay Holding by Elisabeth Sanxay Holding 𝙓
Ernest K. Gann by Ernest K. Gann 𝙓
Le Blé en herbe by Colette 𝙓
Bleak House by Charles Dickens 𝙓
Bless the Beasts and Children (novel) by Glendon Swarthout 𝙓
The Blessing (novel) by Nancy Mitford 𝙓
Blood and Chocolate (novel) by Annette Curtis Klause ✔️
Elliott Arnold by Elliott Arnold 𝙓
Luke Short (writer) by Luke Short (writer) 𝙓
Blood on the Moon (novel) by James Ellroy ✔️
John Hawkes (novelist) by John Hawkes (novelist) 𝙓
Blood Work (novel) by Michael Connelly 𝙓
Colin Thiele bibliography by Colin Thiele 𝙓
The Blue Lagoon (novel) by Henry De Vere Stacpoole 𝙓
The Body (novel) by Richard Sapir 𝙓
The Body Snatchers by Jack Finney ✔️
Bombaiyer Bombete (novel) (page does not exist) by Satyajit Ray 𝙓
The Bonfire of the Vanities by Tom Wolfe ✔️
Bonjour Tristesse by Françoise Sagan 𝙓
Boogiepop and Others by Kouhei Kadono 𝙓
The Book of Daniel (novel) by E. L. Doctorow ✔️
Lothar-Günther Buchheim 

The Deceivers by John Masters 𝙓
The Deep (Peter Benchley novel) by Peter Benchley 𝙓
The Deep End of the Ocean by Jacquelyn Mitchard ✔️
Deliverance (novel) by James Dickey ✔️
Deluge (novel) by S. Fowler Wright 𝙓
The Deluge (novel) by Henryk Sienkiewicz 𝙓
Demon Seed (novel) by Dean Koontz ✔️
The Living and the Dead (Boileau-Narcejac novel) by Boileau-Narcejac 𝙓
Derailed (novel) by James Siegel ✔️
Desert of the Heart by Jane Rule ✔️
The Desperate Hours (Hayes novel) by Joseph Hayes (author) 𝙓
Destry Rides Again (novel) by Max Brand ✔️
The Detective (novel) by Roderick Thorp ✔️
Detective (novel) by Arthur Hailey 𝙓
The Devil Rides Out by Dennis Wheatley 𝙓
The Devil Wears Prada (novel) by Lauren Weisberger ✔️
The Devil's Advocate (West novel) by Morris West ✔️
The Devil's Advocate (Neiderman novel) (page does not exist) by Andrew Neiderman 𝙓
The Possessed (novel) by Fyodor Dostoyevsky 𝙓
Le Diable au corps (novel) by Raymond Radiguet 𝙓
Diamonds Are Forever (novel) by Ian Fleming 𝙓
Dirty Weeke

The Heart of the Matter by Graham Greene 𝙓
The Heaven Sword and Dragon Saber by Jin Yong ✔️
Heidi by Johanna Spyri 𝙓
Hell House (novel) by Richard Matheson ✔️
The Hellbound Heart by Clive Barker ✔️
The Help by Kathryn Stockett ✔️
The High and the Mighty (novel) by Ernest K. Gann 𝙓
High Fidelity (novel) by Nick Hornby ✔️
The History of Mr. Polly by H. G. Wells 𝙓
The History of Tom Jones, a Foundling by Henry Fielding 𝙓
A History of Violence by John Wagner 𝙓
The Hitchhiker's Guide to the Galaxy (novel) by Douglas Adams ✔️
The Hobbit by J. R. R. Tolkien ✔️
The Holcroft Covenant by Robert Ludlum ✔️
The Holy Innocents (Adair novel) by Gilbert Adair 𝙓
Hombre (novel) by Elmore Leonard 𝙓
The Home and the World by Rabindranath Tagore 𝙓
The Honorary Consul by Graham Greene ✔️
Horrid Henry by Francesca Simon 𝙓
Horseman, Pass By by Larry McMurtry 𝙓
The Host (novel) by Stephenie Meyer ✔️
The Hound of the Baskervilles by Arthur Conan Doyle 𝙓
The Hours (novel) by Michael Cunningham ✔️
How Green Was M

The Naked Face by Sidney Sheldon ✔️
Naked Lunch by William S. Burroughs ✔️
The Natural by Bernard Malamud ✔️
National Velvet by Enid Bagnold 𝙓
Nevada (Zane Grey novel) by Zane Grey 𝙓
Never Cry Wolf by Farley Mowat ✔️
The Neverending Story by Michael Ende ✔️
Nicholas Nickleby by Charles Dickens ✔️
The Nick Adams Stories by Ernest Hemingway 𝙓
Night and the City (novel) by Gerald Kersh ✔️
The Night of the Hunter (novel) by Davis Grubb 𝙓
Night Watch (Lukyanenko novel) by Sergey Lukyanenko 𝙓
Nightmare Alley (novel) by William Lindsay Gresham 𝙓
Nights in Rodanthe (novel) by Nicholas Sparks ✔️
Nightwing (novel) by Martin Cruz Smith 𝙓
Nineteen Eighty-Four by George Orwell ✔️
No Country for Old Men by Cormac McCarthy ✔️
No Orchids for Miss Blandish (novel) by James Hadley Chase 𝙓
Northern Lights (novel) by Philip Pullman ✔️
Northwest Passage (novel) by Kenneth Roberts (author) ✔️
Nostromo by Joseph Conrad 𝙓
The Notebook (novel) by Nicholas Sparks ✔️
Nothing Lasts Forever (1979 novel) by Roderic

Small Vices by Robert B. Parker ✔️
Snow Angels (novel) by Stewart O'Nan 𝙓
Solaris (novel) by Stanislaw Lem ✔️
Something Wicked This Way Comes (novel) by Ray Bradbury ✔️
The Song of Bernadette (novel) by Franz Werfel 𝙓
Sons and Lovers by D. H. Lawrence 𝙓
Sour Sweet by Timothy Mo ✔️
Fatelessness by Imre Kertész ✔️
The Sound and the Fury by William Faulkner ✔️
Space (novel) by James A. Michener ✔️
The Space Vampires by Colin Wilson ✔️
The Spanish Gardener by A. J. Cronin ✔️
Spartacus (Fast novel) by Howard Fast ✔️
Sphere (novel) by Michael Crichton ✔️
Spider (novel) by Patrick McGrath (novelist) ✔️
fr:Spirales by fr:Christopher Frank 𝙓
Split Images by Elmore Leonard ✔️
The Spoilers (Beach novel) by Rex Beach 𝙓
The Spook Who Sat by the Door (novel) by Sam Greenlee ✔️
Spy Story (novel) by Len Deighton ✔️
The Spy Who Came in from the Cold by John le Carré ✔️
The Spy Who Loved Me (novel) by Ian Fleming 𝙓
The Stand by Stephen King ✔️
An Ancient Tale (novel) by Józef Ignacy Kraszewski 𝙓
Stardus

In [47]:
add_valid_books_to_dict()

Usable books: 1, Broken books: -1, Total books: 0
Usable books: 63, Broken books: 37, Total books: 100
Usable books: 119, Broken books: 81, Total books: 200
Usable books: 170, Broken books: 130, Total books: 300
Usable books: 206, Broken books: 194, Total books: 400
Usable books: 237, Broken books: 263, Total books: 500
Usable books: 286, Broken books: 314, Total books: 600
Usable books: 343, Broken books: 357, Total books: 700
Usable books: 397, Broken books: 403, Total books: 800
Usable books: 448, Broken books: 452, Total books: 900
Usable books: 500, Broken books: 500, Total books: 1000
Usable books: 547, Broken books: 553, Total books: 1100
Usable books: 600, Broken books: 600, Total books: 1200
Usable books: 622, Broken books: 600, Total books: 1237


In [48]:
print(len(book_movie_dict), len(movie_book_dict))

622 1834


### Data after Lv 1 enrichment 

In [46]:
movie_book_dict['https://en.wikipedia.org/wiki/A_Clockwork_Orange_(film)']

{'movie_title': 'A Clockwork Orange (film)',
 'book_title': 'A Clockwork Orange (novel)',
 'imdb_no': '',
 'usable': '',
 'isbn': '0434098000',
 'oclc': '4205836'}

In [49]:
book_movie_dict['A Clockwork Orange (1962)']

{'author': 'Anthony Burgess',
 'book_title': 'A Clockwork Orange (novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/A_Clockwork_Orange_(novel)',
 'count': 0,
 'adaptations': [],
 'isbn': '0434098000',
 'oclc': '4205836',
 'valid_identifier': True}

### Get IMDB numbers for movies provided by Wikipedia

In [53]:
get_existing_movie_imdb_nos()

The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn ✔️
The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn 𝙓
Adventures of Huckleberry Finn ✔️
Adventures of Huckleberry Finn ✔️
Adventures of Huckleberry Finn ✔️
The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn ✔️
The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn 𝙓
Adventures of Huckleberry Finn 𝙓
The Adventures of Tom Sawyer ✔️
The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn ✔️
The Adventures of Tom Sawyer ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio 𝙓
The Adventures of Pinocchio 𝙓
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Adventures of Pinocchio ✔️
The Advent

Little Women ✔️
Little Women ✔️
Little Women 𝙓
Little Women ✔️
The Littles ✔️
The Littles ✔️
The Littles ✔️
The Lorax ✔️
The Lorax ✔️
Lottie and Lisa ✔️
Lottie and Lisa 𝙓
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Lottie and Lisa ✔️
Madicken ✔️
Madicken ✔️
The Wonderful Wizard of Oz ✔️
The Wonderful Wizard of Oz ✔️
The Marvelous Land of Oz ✔️
The Wonderful Wizard of Oz ✔️
The Wonderful Wizard of Oz ✔️
The Wonderful Wizard of Oz ✔️
The Wonderful Wizard of Oz ✔️
Mary Poppins (book series) ✔️
Mary Poppins (book series) ✔️
Mother Carey's Chickens (novel) ✔️
Mother Carey's Chickens (novel) ✔️
Mrs. Frisby and the Rats of NIMH ✔️
Mrs. Frisby and the Rats of NIMH ✔️
My Friend Flicka ✔️
My Friend Flicka ✔️
My Friend Flicka ✔️
My Friend Flicka ✔️
My Friend Flicka ✔️
Nancy Drew 𝙓
Nancy Drew ✔️
Nancy Drew ✔️
Nancy Drew ✔️
Nancy Drew ✔️
The Neverending Story ✔️
The Neverending Story ✔️
The Neverending Story ✔️

A Wrinkle in Time ✔️
A Wrinkle in Time ✔️
The Yearling ✔️
The Yearling 𝙓
Zenon: Girl of the 21st Century ✔️
Zenon: Girl of the 21st Century ✔️
Zenon: Girl of the 21st Century ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The Diary of a Young Girl ✔️
The A.B.C. Murders ✔️
The A.B.C. Murders ✔️
The Adventures of Tom Sawyer ✔️
Adventures of Huckleberry Finn 𝙓
The Adventures of Pinocchio ✔️
The Adventures of Sherlock Holmes ✔️
The Adventures of Sherlock Holmes 𝙓
The Adventures of Sherlock Holmes ✔️
The Adventures of Sherlock Holmes ✔️
The Adventures of Sherlock Holmes 𝙓
The Age of Innocence 𝙓
The Age of Innocence 𝙓
The Age of Innocence 𝙓
The Age of Innocence ✔️
Aimez-vous Brahms? ✔️
Aimez-vous Brahms? ✔️
Airport (novel

Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
Frankenstein ✔️
The Friendly Persuasion ✔️
The Friendly Persuasion ✔️
From the Earth to the Moon ✔️
From the Earth to the Moon ✔️
From the Earth to the Moon ✔️
From the Earth to the Moon 𝙓
Gentlemen Prefer Blondes (novel) ✔️
Gentlemen Prefer Blondes (novel) ✔️
The Getaway (novel) ✔️
The Getaway (novel) ✔️
Gigi (novella) ✔️
Gigi (novella) ✔️
The Girl Who Leapt Through Time ✔️
The Girl Who Leapt Through Time ✔️
The Girl Who Leapt Through Time 𝙓
The Girl Who Leapt Through Time ✔️
The Girl Who Leapt Through Time ✔️
The

The Midwich Cuckoos ✔️
Mine-Haha, or On the Bodily Education of Young Girls ✔️
Mine-Haha, or On the Bodily Education of Young Girls ✔️
The Mirror Crack'd from Side to Side ✔️
The Mirror Crack'd from Side to Side ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Index: 1350, Usable: 1105 --> 81.852%
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misérables ✔️
Les Misér

Solaris (novel) ✔️
Solaris (novel) ✔️
Sons and Lovers ✔️
Sons and Lovers ✔️
Sons and Lovers 𝙓
Spartacus (Fast novel) ✔️
Spartacus (Fast novel) ✔️
The Spoilers (Beach novel) ✔️
The Spoilers (Beach novel) ✔️
The Spoilers (Beach novel) ✔️
The Spoilers (Beach novel) ✔️
The Spoilers (Beach novel) ✔️
The Stand ✔️
The Stand 𝙓
Starship Troopers ✔️
Starship Troopers ✔️
Starship Troopers ✔️
The Stepford Wives ✔️
The Stepford Wives ✔️
The Stepford Wives ✔️
The Stepford Wives ✔️
The Stepford Wives ✔️
Stella Dallas (novel) ✔️
Stella Dallas (novel) ✔️
Stella Dallas (novel) ✔️
A Stir of Echoes ✔️
A Stir of Echoes ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde 𝙓
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jekyll and Mr Hyde ✔️
Strange Case of Dr Jeky

In [54]:
movie_book_dict['https://en.wikipedia.org/wiki/A_Clockwork_Orange_(film)']

{'movie_title': 'A Clockwork Orange (film)',
 'book_title': 'A Clockwork Orange (novel)',
 'imdb_no': 'tt0066921',
 'usable': True,
 'isbn': '0434098000',
 'oclc': '4205836'}

### Scrape IMDb lists to add new movies and books to dictionaries

In [56]:
add_all_imdb_lists(imdb_lists)

Working on: Book-to-Film Adaptations - IMDb | https://www.imdb.com/list/ls000989358/
Twilight_(novel) ✔
Lord of the Flies ✔
The Fellowship of the Ring ✔
The Return of the King ✔
The Two Towers ✔
Pride & Prejudice ✔
Sense and Sensibility ✔
Jane Eyre ✔
I Capture the Castle ✔
The Bourne Identity_(novel) ✔
The Firm_(novel) ✔
The Rainmaker_(novel) ✔
The Bourne Supremacy ✔
The Bourne Ultimatum ✔
In My Father's Den ✔
Let the Right One In_(novel) ✔
Let Me In_(novel) ✔
The Far Side of the World ✔
Vanity Fair_(novel) ✔
I Am Number Four ✔
Memoirs of a Geisha ✔
Harry Potter and the Sorcerer's Stone ✔
Harry Potter and the Chamber of Secrets ✔
Harry Potter and the Goblet of Fire ✔
Harry Potter and the Prisoner of Azkaban ✔
Harry Potter and the Order of the Phoenix ✔
Harry Potter and the Half-Blood Prince ✔
Girl with a Pearl Earring_(novel) ✔
Flowers in the Attic ✔
Dracula ✔
The Four Feathers ✔
The Other Boleyn Girl ✔
The Black Dahlia_(novel) ✔
Little Children_(novel) ✔
Bridget Jones's Diary ✔
Bridge

### _De facto_ data size

In [57]:
print(len(book_movie_dict), len(movie_book_dict))

769 2119


## Enrich data using APIs

### Get book relatated metadata from OpenLibrary API and Goodreads APIs 

In [86]:
book_metadata_refresh()

Adventures of Huckleberry Finn (1884) 𝙓
The Adventures of Tom Sawyer (1876) ✔
Alexander and the Terrible ✔
Angus 𝙓
Aquamarine (2001) 𝙓
Arrow to the Sun (1974) ✔
Ballet Shoes (1936) 𝙓
Bambi ✔
Beastly (2007) 𝙓
Because of Winn-Dixie (2000) ✔
The BFG (1982) ✔
Big Fish: A Novel of Mythic Proportions (1998) ✔
The Big Six (1940) 𝙓
Blood and Chocolate (1997) ✔
The Book Thief (2005) 𝙓
The Borrowers (1952) ✔
The Breadwinner (2000) 𝙓
Bridge to Terabithia (1977) ✔
The Brothers Lionheart (1973) ✔
The Butter Battle Book (1984) ✔
The Call of the Wild (1903) 𝙓
The Cat in the Hat (1957) 𝙓
Charlie and the Chocolate Factory (1964) 𝙓
Charlotte's Web (1952) ✔
Chitty Chitty Bang Bang: The Magical Car (1964) 𝙓
The Chocolate War (1974) ✔
Cirque du Freak (2000) Darren Shan 𝙓
The City of Ember (2003) ✔
Cloudy with a Chance of Meatballs (1978) ✔
Confessions of a Teenage Drama Queen (1999) ✔
Coot Club (1934) 𝙓
Coraline (2002) ✔
Daisy-Head Mayzie (1995) ✔
Danny ✔
A Day with Wilbur Robinson (1990) ✔
The Dark is Ris

The Deep End of the Ocean (1996) ✔
Deliverance (1970) ✔
Demon Seed (1973) ✔
Derailed (2003) ✔
Desert of the Heart (1964) ✔
Destry Rides Again (1930) 𝙓
The Detective (1966) ✔
The Devil Wears Prada (2003) ✔
The Devil's Advocate (1954) ✔
Dirty Weekend (1991) ✔
Disclosure (1994) ✔
Divine Secrets of the Ya-Ya Sisterhood (1996) ✔
Do Androids Dream of Electric Sheep? (1967) 𝙓
Doctor Zhivago (1957) ✔
Dog Soldiers (1974) ✔
Dracula (1897) ✔
Dragon Seed (1942) ✔
Dreamcatcher (2001) ✔
Drood (2009) ✔
The Drowning Pool (1950) ✔
The Dying Animal (2001) ✔
The Eagle Has Landed (1975) ✔
East of Eden (1952) ✔
Eaters of the Dead (1976) ✔
The Egyptian (Sinuhe the Egyptian) (1945) ✔
Election (1998) ✔
Elmer Gantry (1927) ✔
Empire of the Sun (1984) ✔
The End of the Affair (1951) 𝙓
Indexed: 300, Updated: 1478, Usable: 492.667%
Enduring Love (1997) 𝙓
The English Patient (1992) ✔
Enigma (1995) ✔
Everybody's All-American (1981) ✔
Everything Is Illuminated (2002) ✔
The Executioners (1957) ✔
Exit to Eden (1984) 𝙓
T

A Stranger Is Watching (1977) ✔
The Street Lawyer (1998) ✔
Strip Tease (1993) ✔
Stupeur et Tremblement (Fear and Trembling) (1999) ✔
The Sum of All Fears (1991) ✔
The Talented Mr. Ripley (1955) ✔
The Terminal Man (1972) ✔
The Tesseract (1998) ✔
Tex (1979) ✔
Texas (1985) ✔
Thin Air (1995) ✔
Time After Time (1979) 𝙓
A Time to Kill (1989) ✔
The Time Traveler's Wife (2003) ✔
Timeline (1999) ✔
Tinker ✔
To the Last Man (1921) ✔
Der Tod in Venedig (Death in Venice) (1912) ✔
Trainspotting (1993) ✔
Trinity's Child (1983) ✔
The Turn of the Screw (1898) 𝙓
The Twins (2000) 𝙓
The Two Towers (1954) ✔
Union Street (1982) ✔
Tuntematon sotilas (The Unknown Soldier) (1954) 𝙓
Valley of the Dolls (1966) ✔
Vampire$ (1990) ✔
Vanity Fair (1847–1848) 𝙓
Vigil in the Night (1939) ✔
The Virginian: A Horseman of the Plains (1902) 𝙓
Vision Quest (1979) ✔
A Walk to Remember (1999) ✔
Walkabout (a.k.a. The Children) (1959) ✔
Walking Shadow (1994) ✔
Water for Elephants (2006) ✔
What Dreams May Come (1978) 𝙓
Indexed: 6

In [178]:
clean_gc_authors()

Adventures of Huckleberry Finn (1884) 𝙓
The Adventures of Tom Sawyer (1876) 𝙓
Alexander and the Terrible 𝙓
Angus 𝙓
Aquamarine (2001) 𝙓
Arrow to the Sun (1974) 𝙓
Ballet Shoes (1936) 𝙓
Bambi 𝙓
Beastly (2007) 𝙓
Because of Winn-Dixie (2000) 𝙓
The BFG (1982) 𝙓
Big Fish: A Novel of Mythic Proportions (1998) 𝙓
The Big Six (1940) 𝙓
Blood and Chocolate (1997) 𝙓
The Book Thief (2005) 𝙓
The Borrowers (1952) 𝙓
The Breadwinner (2000) 𝙓
Bridge to Terabithia (1977) 𝙓
The Brothers Lionheart (1973) 𝙓
The Butter Battle Book (1984) 𝙓
The Call of the Wild (1903) 𝙓
The Cat in the Hat (1957) 𝙓
Charlie and the Chocolate Factory (1964) 𝙓
Charlotte's Web (1952) 𝙓
Chitty Chitty Bang Bang: The Magical Car (1964) 𝙓
The Chocolate War (1974) 𝙓
Cirque du Freak (2000) Darren Shan 𝙓
The City of Ember (2003) 𝙓
Cloudy with a Chance of Meatballs (1978) 𝙓
Confessions of a Teenage Drama Queen (1999) 𝙓
Coot Club (1934) 𝙓
Coraline (2002) 𝙓
Daisy-Head Mayzie (1995) 𝙓
Danny 𝙓
A Day with Wilbur Robinson (1990) 𝙓
The Dark is Ris

### Get book relatated metadata from IMDb API 

In [177]:
movie_metadata_refresh()

Huck and Tom ✔
Huckleberry Finn (1920 film) ✔
Huckleberry Finn (1931 film) ✔
Huck Finn (1937 film) (page does not exist) 𝙓
The Adventures of Huckleberry Finn (1939 film) ✔
The Adventures of Huckleberry Finn (1955 film) ✔
The Adventures of Huckleberry Finn (1960 film) ✔
Huckleberry Finn (1974 film) ✔
Hopelessly Lost ✔
Huckleberry Finn (1975 film) ✔
The Adventures of Huckleberry Finn (1981 film) (page does not exist) 𝙓
Adventures of Huckleberry Finn (1985 film) (page does not exist) 𝙓
The Adventures of Mark Twain (1986 film) ✔
Back to Hannibal: The Return of Tom Sawyer and Huckleberry Finn ✔
The Adventures of Huck Finn (1993 film) ✔
Tom and Huck ✔
The Adventures of Pinocchio (1936 film) ✔
Pinocchio (1940 film) ✔
Pinocchio (1957 TV-musical) ✔
The Adventures of Buratino (1959 film) ✔
Pinocchio in Outer Space ✔
Pinocchio (1968 film) ✔
The Adventures of Pinocchio (1972 film) ✔
The Adventures of Buratino (1975 film) ✔
Pinocchio (1976 TV-musical) (page does not exist) 𝙓
The Adventures of Pinoc

Brink! ✔
Harriet the Spy (film) ✔
Harriet the Spy: Blog Wars ✔
Heidi (1937 film) ✔
Heidi (1952 film) ✔
Heidi and Peter ✔
Heidi (1959 film) (page does not exist) 𝙓
Heidi (1968 film) ✔
Heidi, Girl of the Alps ✔
Heidi's Song ✔
Heidi (1993 TV miniseries) (page does not exist) 𝙓
Heidi (1996 film) ✔
Heidi (2005 animated film) ✔
Nelvana ✔
Heidi (2005 live-action film) ✔
Paul Marcus 𝙓
Heidi 4 Paws ✔
The Hobbit (1977 film) ✔
The Hobbit (film series) 𝙓
The Hobbit: An Unexpected Journey ✔
The Hobbit: The Desolation of Smaug ✔
The Hobbit: The Battle of the Five Armies ✔
Horton Hatches the Egg 𝙓
Horton Hatches the Egg 𝙓
Horton Hears a Who! (TV special) ✔
Horton Hears a Who! (film) ✔
The Shaggy Dog (1959 film) ✔
The Shaggy D.A. ✔
The Shaggy Dog (2006 film) ✔
How the Grinch Stole Christmas! (TV special) ✔
Halloween is Grinch Night ✔
How the Grinch Stole Christmas! (film) ✔
The Grinch (film) ✔
How To Train Your Dragon (film) ✔
How To Train Your Dragon 2 ✔
How to Train Your Dragon: The Hidden World ✔
O

Around the World in 80 Days (TV miniseries) ✔
Around the World in Eighty Days (novel) (1999) 𝙓
Tweety's High-Flying Adventure ✔
Around the World in 80 Days (2004 film) ✔
Treasure Island (1918) ✔
Treasure Island (1920 film) ✔
Treasure Island (1934 film) ✔
Treasure Island (1950 film) ✔
Long John Silver (film) ✔
Return to Treasure Island (1954 film) ✔
Doubutsu Takarajima ✔
Treasure Island (1972 animated film) ✔
Treasure Island (1972 live-action film) ✔
Treasure Island (1977 TV Mini Series) ✔
Treasure Island (1982 film) ✔
Treasure Island (1985 film) ✔
Return to Treasure Island (TV series) ✔
Treasure Island (1987) ✔
Treasure Island (1988 film) ✔
Treasure Island (1990 film) ✔
Muppet Treasure Island ✔
Treasure Island (1999 film) ✔
Treasure Planet ✔
Pirates of Treasure Island ✔
Treasure Island (2007) ✔
The Three Musketeers in film (1903) 𝙓
The Three Musketeers in film (1911) 𝙓
The Three Musketeers in film (1914) 𝙓
The Three Musketeers in film (1916) 𝙓
Les Trois Mousquetaires ✔
The Three Musket

Anna Karenina (1997 film) ✔
Anna Karenina (2005 film) (page does not exist) 𝙓
Anna Karenina (2012 film) ✔
Anne of Green Gables (1972 TV series) ✔
Buck Rogers (serial) ✔
Buck Rogers in the 25th Century (TV series) ✔
Assunta Spina (1915 film) ✔
Assunta Spina (1930 film) ✔
Assunta Spina (1948 film) ✔
Assunta Spina (2006 film) (page does not exist) 𝙓
At the Earth's Core (film) ✔
Back Street (1932 film) ✔
Back Street (1941 film) ✔
Back Street (1961 film) ✔
Battle Royale (film) ✔
Battle Royale II: Requiem ✔
The Beastmaster (film) ✔
Index: 750, Updated: 593 --> 79%
Beastmaster 2: Through the Portal of Time ✔
Beastmaster III: The Eye of Braxus ✔
La Bestia debe morir ✔
Que la bête meure ✔
Beau Geste (1926 film) ✔
Beau Hunks ✔
Beau Geste (1939 film) ✔
Beau Geste (1966 film) ✔
Follow That Camel ✔
The Last Remake of Beau Geste ✔
Beau Geste (TV series) ✔
Bel Ami (1939 film) ✔
The Private Affairs of Bel Ami ✔
Bel Ami (2005) ✔
Bel Ami (2012 film) ✔
Son of Fury: The Story of Benjamin Blake ✔
Treasure 

Friendly Persuasion (1975 film) ✔
From the Earth to the Moon (film) ✔
The Fabulous Baron Munchausen ✔
Jules Verne's Rocket to the Moon ✔
From the Earth to the Moon (1979 film) (page does not exist) 𝙓
Gentlemen Prefer Blondes (1928 film) ✔
Gentlemen Prefer Blondes (1953 film) ✔
The Getaway (1972 film) ✔
The Getaway (1994 film) ✔
Gigi (1949 film) ✔
Gigi (1958 film) ✔
Toki o Kakeru Shōjo (1983 film) ✔
Toki o Kakeru Shōjo (1997 film) ✔
Morning Musume: Shinshun! Love Stories 𝙓
The Girl Who Leapt Through Time (2006 film) ✔
Time Traveller: The Girl Who Leapt Through Time ✔
Suburban Girl ✔
The Girls' Guide to Hunting and Fishing (novel) 𝙓
The Go-Between (1971 film) ✔
The Go-Between (2015 film) ✔
The Godfather ✔
The Godfather Part II ✔
The Godfather Part III ✔
Gettysburg (1993 film) ✔
Gods and Generals (film) ✔
The Golden Bowl (1972) ✔
The Golden Bowl (film) ✔
Goodbye, Mr. Chips (1939 film) ✔
Goodbye, Mr. Chips (1969 film) ✔
Goodbye, Mr. Chips (1984) ✔
Index: 1050, Updated: 862 --> 82%
Goodbye,

The Lost World (1925 film) ✔
The Lost World (1960 film) ✔
The Lost World (1992 film) ✔
Return to the Lost World ✔
The Lost World (1998 film) ✔
Sir Arthur Conan Doyle's The Lost World 𝙓
The Lost World (2001 film) ✔
King of the Lost World ✔
Magnificent Obsession (1935 film) ✔
Magnificent Obsession (1954 film) ✔
Master of the World (1961 film) ✔
Master of the World (novel) (1961) 𝙓
The Maltese Falcon (1931 film) ✔
Satan Met a Lady ✔
The Maltese Falcon (1941 film) ✔
The Black Bird ✔
Man on Fire (1987 film) ✔
Man on Fire (2004 film) ✔
The Man Who Fell to Earth (film) ✔
The Man Who Fell to Earth (TV) 𝙓
The Man Who Fell to Earth (film) ✔
The Manchurian Candidate (1962 film) ✔
The Manchurian Candidate (2004 film) ✔
The Mayor of Casterbridge (1921 film) ✔
The Mayor of Casterbridge (1978) 𝙓
The Mayor of Casterbridge (2001) 𝙓
The Claim (2000 film) ✔
McTeague (film) ✔
Greed (1924 film) ✔
Merton of the Movies (1924 film) ✔
Make Me a Star (film) ✔
Merton of the Movies (1947 film) ✔
Tony Rome ✔
Lady 

Manhunter (film) ✔
Red Dragon (2002 film) ✔
The Rescuers ✔
The Rescuers Down Under ✔
The Story of the Great Heroes 𝙓
The Brave Archer and His Mate ✔
Little Dragon Maiden ✔
The Return of the King 𝙓
The Lord of the Rings: The Return of the King ✔
Return of the Living Dead (film series) ✔
The Return of the Living Dead ✔
Return of the Living Dead Part II ✔
Return of the Living Dead 3 ✔
Return of the Living Dead: Necropolis ✔
Return of the Living Dead: Rave to the Grave ✔
The Return of the Native (1994) ✔
The Return of the Native (2010) ✔
Return to Paradise (1953 film) ✔
Until They Sail ✔
The Riddle of the Sands (film) ✔
The Riddle of the Sands (1984) ✔
Riders of the Purple Sage (1918) 𝙓
Riders of the Purple Sage (1925) 𝙓
Riders of the Purple Sage (1931) 𝙓
Riders of the Purple Sage (1941) 𝙓
Riders of the Purple Sage (1996) 𝙓
Riverworld (2003 film) ✔
Riverworld (2010 film) ✔
The Crimson Rivers ✔
Crimson Rivers II: Angels of the Apocalypse ✔
Rosemary's Baby (film) ✔
Look What's Happened to Ro

War of the Worlds (2005 film) ✔
H. G. Wells' The War of the Worlds (Timothy Hines film) ✔
H. G. Wells' War of the Worlds (David Michael Latt film) ✔
War of the Worlds 2: The Next Wave ✔
The Curse of the Werewolf ✔
Legend of the Werewolf ✔
The Wind in the Willows (2010) ✔
The Wings of the Dove (1981 film) ✔
The Wings of the Dove (1997 film) ✔
The Woman in Black (1989 film) ✔
The Woman in Black (2012 film) ✔
The Woman in White (novel) (1912) 𝙓
The Woman in White (novel) (1917) 𝙓
The Woman in White (novel) (1929) 𝙓
Crimes at the Dark House ✔
The Woman in White (1948 film) ✔
The Woman in White (novel) (1982) 𝙓
The Woman in White (novel) (1982) 𝙓
Wuthering Heights (1939 film) ✔
Wuthering Heights (1954 film) ✔
Wuthering Heights (1970 film) ✔
Emily Brontë's Wuthering Heights ✔
Wuthering Heights (2003 film) ✔
Twilight ✔
The Twilight Saga: New Moon ✔
The Twilight Saga: Eclipse ✔
Lord of the Flies ✔
The Lord of the Rings: The Fellowship of the Ring ✔
The Lord of the Rings: The Return of the King

In [179]:
lowercase_everything()

Books lower-cased.
Movies lower-cased.


## Create unique pairing IDs for every movie and book

In [180]:
match_ids()

Books updated.
Movies updated.


### Deep copy dictionaries

In [181]:
#DON'T RUN AGAIN
safe_book_movie_dict = copy.deepcopy(book_movie_dict)
id(safe_book_movie_dict), id(book_movie_dict)

(4805342552, 4586808808)

In [182]:
#DON'T RUN AGAINabs
safe_movie_book_dict = copy.deepcopy(movie_book_dict)
id(safe_movie_book_dict), id(movie_book_dict)

(4805307200, 4674427712)

## Data enrichment complete!

In [183]:
movie_book_dict['https://en.wikipedia.org/wiki/A_Clockwork_Orange_(film)']

{'movie_title': 'A Clockwork Orange (film)',
 'book_title': 'A Clockwork Orange (novel)',
 'imdb_no': 'tt0066921',
 'usable': True,
 'isbn': '0434098000',
 'oclc': '4205836',
 'imdb_title': 'A Clockwork Orange',
 'imdb_actors': ['malcolm mcdowell',
  'patrick magee',
  'michael bates',
  'warren clarke'],
 'imdb_writers': ['stanley kubrick', 'anthony burgess'],
 'imdb_genres': ['crime', 'drama', 'sci-fi'],
 'imdb_year': '1971',
 'imdb_rated': 'R',
 'imdb_released': '02 Feb 1972',
 'imdb_runtime': '136 min',
 'imdb_imdbrating': '8.3',
 'imdb_imdbvotes': '691,997',
 'budget': '2200000',
 'gross_usa': '26589355',
 'metadata_updated': True,
 'book_id': 'b_0434098000_4205836',
 'movie_id': 'm_0434098000_4205836_tt0066921'}

In [184]:
book_movie_dict['A Clockwork Orange_(novel)']

{'author': 'Anthony Burgess',
 'book_title': 'A Clockwork Orange_(novel)',
 'book_wiki_url': 'https://en.wikipedia.org/wiki/A Clockwork Orange_(novel)',
 'count': 1,
 'adaptations': ['A Clockwork Orange'],
 'isbn': '0434098000',
 'oclc': '4205836',
 'valid_identifier': True,
 'number_of_pages': 195,
 'subject_places': ['England', 'Korova Milk Bar'],
 'subjects': ['Internet Archive Wishlist',
  'bible',
  'aversion therapy',
  'unintended consequences',
  "Beethoven's Fifth Symphony",
  'gang rape',
  'psychiatric hospitals',
  'maturity',
  'metanoia',
  'assault',
  'joyriding',
  'rape',
  'metafiction',
  'orgasm',
  'novella',
  'antisocial personality disorde',
  'classical music',
  'black comedy',
  'argot',
  'Nadsat',
  'dystopias',
  'satire',
  'literature',
  'Juvenile delinquents',
  'Science fiction',
  'Rehabilitation',
  'Fiction',
  'Protected DAISY',
  'Accessible book',
  'In library',
  'Teenage boys',
  'Criminals',
  'Sátira',
  'Muchachos adolescentes',
  'Critic

In [185]:
movie_book_dict['https://en.wikipedia.org/wiki/Cape_Fear_(1962_film)']

{'movie_title': 'Cape Fear (1962 film)',
 'book_title': 'The Executioners (MacDonald novel)',
 'imdb_no': 'tt0055824',
 'usable': True,
 'isbn': '0449131904',
 'oclc': '',
 'imdb_title': 'Cape Fear',
 'imdb_actors': ['gregory peck',
  'robert mitchum',
  'polly bergen',
  'lori martin'],
 'imdb_writers': ['john d. macdonald', 'james r. webb'],
 'imdb_genres': ['drama', 'thriller'],
 'imdb_year': '1962',
 'imdb_rated': 'Not Rated',
 'imdb_released': '21 Aug 1962',
 'imdb_runtime': '106 min',
 'imdb_imdbrating': '7.8',
 'imdb_imdbvotes': '23,626',
 'budget': '3000000',
 'metadata_updated': True,
 'book_id': 'b_0449131904_',
 'movie_id': 'm_0449131904__tt0055824'}

In [186]:
movie_book_dict['https://en.wikipedia.org/wiki/Cape_Fear_(1991_film)']

{'movie_title': 'Cape Fear (1991 film)',
 'book_title': 'The Executioners (MacDonald novel)',
 'imdb_no': 'tt0101540',
 'usable': True,
 'isbn': '0449131904',
 'oclc': '',
 'imdb_title': 'Cape Fear',
 'imdb_actors': ['robert de niro',
  'nick nolte',
  'jessica lange',
  'juliette lewis'],
 'imdb_writers': ['john d. macdonald', 'james r. webb', 'wesley strick'],
 'imdb_genres': ['crime', 'thriller'],
 'imdb_year': '1991',
 'imdb_rated': 'R',
 'imdb_released': '15 Nov 1991',
 'imdb_runtime': '128 min',
 'imdb_imdbrating': '7.3',
 'imdb_imdbvotes': '153,874',
 'budget': '35000000',
 'gross_usa': '79100000',
 'cumulative_worldwide_gross': '103200000',
 'metadata_updated': True,
 'book_id': 'b_0449131904_',
 'movie_id': 'm_0449131904__tt0101540'}

## Converting to dataframes

In [231]:
book_movie = pd.DataFrame.from_dict(data = book_movie_dict, orient='index')
book_movie.set_index('book_id', inplace = True)
book_movie.head()

,author,book_title,book_wiki_url,count,adaptations,isbn,oclc,valid_identifier,metadata_updated,number_of_pages,subject_places,subjects,publish_date,publish_places,goodreads,gc_title,gc_language_code,gc_average_rating,gc_format,gc_ratings_count,gc_text_reviews_count,title,gc_authors
book_id,,,,,,,,,,,,,,,,,,,,,,,
b_9780385007511_848489,Stephen King,'Salem's Lot,https://en.wikipedia.org/wiki/%27Salem%27s_Lot,0,[],9780385007511,848489,True,True,427.0,"[Maine, U.S.A., Jerusalem's Lot, Marsten House...","[Catholic Church, arson, Great Depression, hit...",1976,[New York],1589604,Salem's Lot,en-US,4.01,Hardcover,443,65,'salem's Lot,NaN
b_0684819066_38377745,,A Beautiful Mind_(book),https://en.wikipedia.org/wiki/A Beautiful Mind...,1,[A Beautiful Mind],0684819066,38377745,True,True,459.0,[United States],"[Mathematicians, Biography]",1998,[New York],NaN,NaN,NaN,NaN,NaN,NaN,NaN,A beautiful mind,NaN
b_0434098000_4205836,Anthony Burgess,A Clockwork Orange (novel),https://en.wikipedia.org/wiki/A_Clockwork_Oran...,0,[],0434098000,4205836,True,True,195.0,"[England, Korova Milk Bar]","[Internet Archive Wishlist, bible, aversion th...",1962,[London],101194,A Clockwork Orange,eng,3.99,Hardcover,142,17,A clockwork orange,NaN
b_0434098000_4205836,Anthony Burgess,A Clockwork Orange_(novel),https://en.wikipedia.org/wiki/A Clockwork Oran...,1,[A Clockwork Orange],0434098000,4205836,True,True,195.0,"[England, Korova Milk Bar]","[Internet Archive Wishlist, bible, aversion th...",1962,[London],101194,A Clockwork Orange,eng,3.99,Hardcover,142,17,A clockwork orange,NaN
b_0064433390_30559641,William Joyce,A Day with Wilbur Robinson,https://en.wikipedia.org/wiki/A_Day_with_Wilbu...,0,[],0064433390,30559641,True,True,32.0,NaN,"[In library, Humorous stories]","September 30, 1993",NaN,566625,A Day with Wilbur Robinson,None,4.03,Paperback,647,81,A Day with Wilbur Robinson,NaN


We eliminate `broken` movies.

In [232]:
movie_book = pd.DataFrame.from_dict(data = movie_book_dict, orient='index')
movie_book = movie_book.loc[movie_book['book_id'] != 'b_broken_broken']
movie_book = movie_book.loc[movie_book['imdb_no'] != 'broken']
movie_book.set_index('movie_id', inplace = True)
movie_book.head(5)

,movie_title,book_title,imdb_no,usable,isbn,oclc,imdb_title,imdb_actors,imdb_writers,imdb_genres,imdb_year,imdb_rated,imdb_released,imdb_runtime,imdb_imdbrating,imdb_imdbvotes,metadata_updated,book_id,budget,gross_usa,cumulative_worldwide_gross,opening_weekend_usa
movie_id,,,,,,,,,,,,,,,,,,,,,,
m_9780385007511_848489_tt0355987,'Salem's Lot (2004 TV miniseries),'Salem's Lot,tt0355987,True,9780385007511,848489,Salem's Lot,"[rob lowe, andre braugher, donald sutherland, ...",[n/a],"[drama, horror, mystery, thriller]",2004,TV-14,20 Jun 2004,181 min,6.2,"9,045",True,b_9780385007511_848489,25000000,NaN,NaN,NaN
m__470015866_tt0048918,1984 (1956 film),Nineteen Eighty-Four,tt0048918,True,,470015866,1984,"[edmond o'brien, michael redgrave, jan sterlin...","[george orwell, william templeton, ralph gilbe...","[drama, sci-fi]",1956,Not Rated,01 Sep 1956,90 min,7.0,"2,779",True,b__470015866,NaN,NaN,NaN,NaN
m_0060250488__tt0202179,ABC Afterschool Special,Freaky Friday,tt0202179,True,0060250488,,ABC Afterschool Specials,"[lance kerwin, samaria graham, mara hobel, ale...",[guy fraumeni],"[adventure, comedy, drama]",1972–1997,N/A,04 Oct 1972,60 min,6.9,611,True,b_0060250488_,NaN,NaN,NaN,NaN
m_0434098000_4205836_tt0066921,A Clockwork Orange (film),A Clockwork Orange (novel),tt0066921,True,0434098000,4205836,A Clockwork Orange,"[malcolm mcdowell, patrick magee, michael bate...","[stanley kubrick, anthony burgess]","[crime, drama, sci-fi]",1971,R,02 Feb 1972,136 min,8.3,"691,997",True,b_0434098000_4205836,2200000,26589355,NaN,NaN
m_0486440885__tt0061270,A Journey to the Center of the Earth (1993),Voyage au centre de la Terre,tt0061270,True,0486440885,,Journey to the Center of the Earth,"[ted knight, pat harrington jr., jane webb]",[n/a],"[animation, family, fantasy]",1967–1969,N/A,09 Sep 1967,30 min,7.8,67,True,b_0486440885_,NaN,NaN,NaN,NaN


### Export to CSV

In [234]:
#Export to csv
book_movie.to_csv('data/books.csv')
movie_book.to_csv('data/movies.csv')

Data is finally obtained! Now we can start the real project. 